In [21]:
import pandas as pd


# I Importation des données

In [22]:
X_train = pd.read_csv("../data/X_train.csv")
y_train = pd.read_csv("../data/y_train.csv")
X_test = pd.read_csv("../data/X_test.csv")
y_test = pd.read_csv("../data/y_test.csv")

X_train = X_train.rename(columns={"1st_Flr_SF": "Frst_Flr_SF"})
X_test = X_test.rename(columns={"1st_Flr_SF": "Frst_Flr_SF"})

# Pipeline et modèle

Il ne faut selectionner que 10 features, on supprime Neighborhood qui est trop complexe à traiter, ainsi que bsmt Qual et garage Finish qui sont trop proches d'autres variables

## Transformation du data set

In [23]:
numeric_features = ["Year Built", "Total Bsmt SF", "1st Flr SF", "Gr Liv Area", "Garage Area", "Overall Qual", "Full Bath"]
ordinal_features = [ "Exter Qual",  "Kitchen Qual"]
cat_feature = ["Neighborhood"]
all_col = numeric_features.copy()
all_col.extend(ordinal_features)
all_col.extend(cat_feature)



In [24]:
def transformation_X(X):
       X = X[all_col]
       X.columns = ['Year_Built', 'Total_Bsmt_SF', 'Frst_Flr_SF', 'Gr_Liv_Area','Garage_Area', 'Overall_Qual', 'Full_Bath', 'Exter_Qual',
              'Kitchen_Qual', 'Neighborhood']
       return X

In [25]:
X_train = transformation_X(X_train)
X_test = transformation_X(X_test)

## Modèle

In [26]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.preprocessing import OrdinalEncoder

numeric_transformer = SimpleImputer()

exter_cat = [ 'Po', 'Fa','TA', 'Gd','Ex']
kitchen_cat = [ 'Po', 'Fa','TA', 'Gd',"Ex"]

ordinal_transformer = OrdinalEncoder(categories=[exter_cat, kitchen_cat])

categorical_transformer = OneHotEncoder()



In [27]:
numeric_features = ["Year_Built", "Total_Bsmt_SF", "Frst_Flr_SF", "Gr_Liv_Area", "Garage_Area", "Overall_Qual", "Full_Bath"]
ordinal_features = [ "Exter_Qual",  "Kitchen_Qual"]
cat_feature = ["Neighborhood"]

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('ord', ordinal_transformer, ordinal_features),
        ('cat', categorical_transformer, cat_feature)
    ]
)

In [28]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()

In [29]:
import numpy as np
from sklearn.metrics import mean_absolute_error

pipe = Pipeline([
     ('preprocessor', preprocessor),
     ('reg', reg)
])


pipe.fit(X_train, y_train) 

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  ['Year_Built',
                                                   'Total_Bsmt_SF',
                                                   'Frst_Flr_SF', 'Gr_Liv_Area',
                                                   'Garage_Area',
                                                   'Overall_Qual',
                                                   'Full_Bath']),
                                                 ('ord',
                                                  OrdinalEncoder(categories=[['Po',
                                                                              'Fa',
                                                                              'TA',
                                                                              'Gd',
                                                                              'Ex'],
  

In [30]:
X_test

,Year_Built,Total_Bsmt_SF,Frst_Flr_SF,Gr_Liv_Area,Garage_Area,Overall_Qual,Full_Bath,Exter_Qual,Kitchen_Qual,Neighborhood
0,1925,588.0,833,1666,228.0,8,1,TA,Gd,OldTown
1,1972,494.0,494,1030,264.0,6,1,TA,TA,BrDale
2,2003,796.0,806,1724,616.0,7,2,Gd,Gd,Edwards
3,1994,990.0,990,990,0.0,4,1,TA,TA,CollgCr
4,1926,894.0,919,919,195.0,6,1,TA,TA,IDOTRR
...,...,...,...,...,...,...,...,...,...,...
581,2003,868.0,1146,1146,438.0,7,2,Gd,Gd,Timber
582,2004,2492.0,2492,2492,949.0,10,2,Ex,Ex,NridgHt
583,2003,1405.0,1405,1405,478.0,7,2,Gd,Gd,NridgHt
584,1967,936.0,936,936,384.0,5,1,TA,TA,Edwards


In [31]:
from sklearn.metrics import mean_absolute_error
pipe.score(X_test,y_test)

predict_train  = pipe.predict(X_train)
predict_test  = pipe.predict(X_test)

# Root Mean Squared Error on train and test date
print('MAE on train data: ', mean_absolute_error(y_train, predict_train))
print('MAE on test data: ',  mean_absolute_error(y_test, predict_test))



MAE on train data:  20267.0928504425
MAE on test data:  21270.49478541006


pipe['reg'].get_feature_names_out

In [32]:
# import pickle
# filename = 'finalized_model.sav'
# pickle.dump(search, open(filename, 'wb'))

In [33]:
from joblib import dump
dump(pipe, "../fit/model.joblib")

['../fit/model.joblib']